# Genetic algo for lighthouse sensor distribution on arbitrary mesh

In [1]:
import numpy as np
from stl import mesh as meshstl
from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt
from plotmesh import plot_mesh
import math
import random
from pyquaternion import Quaternion
from scipy.linalg import qr
import roslib
import rospy
import math
import tf

In [2]:
stl_file = 'roboy_models/TestCube/stls/sphere1m.stl'

In [3]:
#Move Lighthouses to
translationLH1 = [-2.,0,2.]
quat1 = Quaternion(axis=[0,0,1],angle=-np.pi / 2)

global LH1 
LH1 = (translationLH1, quat1)

translationLH2 = [2,0.,2.]
quat2 = Quaternion(axis=[0,0,1], angle=np.pi / 2)

global LH2
LH2 = (translationLH2, quat2)

print(LH1); print(LH2)

([-2.0, 0, 2.0], Quaternion(0.70710678118654757, -0.0, -0.0, -0.70710678118654746))
([2, 0.0, 2.0], Quaternion(0.70710678118654757, 0.0, 0.0, 0.70710678118654746))


In [4]:
from visualization_msgs.msg import Marker, MarkerArray
from data.trafomatrix import get_random_trafomatrix

scale = 0.01
duration = 1
pos = [0,0,0]
orientationMesh = Quaternion(axis=(1,0,0),angle = np.pi*0)

rospy.init_node('fixed_tf_broadcaster')

#Visualization mesh
topic = 'visualization_marker'
publisher = rospy.Publisher(topic, Marker, queue_size=1)
rate2 = rospy.Rate(1)

In [5]:
while not rospy.is_shutdown():
    
    for i in range(3):
        mesh = Marker()
        mesh.header.frame_id = 'world';
        mesh.ns = 'ns';
        mesh.type = Marker.MESH_RESOURCE
        mesh.color.r = 1.0
        mesh.color.g = 1.0
        mesh.color.b = 1.0
        mesh.color.a = 0.9
        mesh.scale.x = scale
        mesh.scale.y = scale
        mesh.scale.z = scale
        mesh.lifetime = rospy.Duration(-1);
        mesh.header.stamp = rospy.Time.now()
        mesh.action = Marker.ADD
        mesh.id = 10000000;
        mesh.pose.position.x = pos[0];
        mesh.pose.position.y = pos[1];
        mesh.pose.position.z = pos[2];
        mesh.pose.orientation.x = orientationMesh[1];
        mesh.pose.orientation.y = orientationMesh[2];
        mesh.pose.orientation.z = orientationMesh[3];
        mesh.pose.orientation.w = orientationMesh[0];
        mesh.mesh_resource = ("package://" + stl_file)
        #mesh.mesh_resource = "package://roboy_models/roboy_2_0_simplified/meshes/CAD/head.stl"

        publisher.publish(mesh);
        rate2.sleep()
        #MeshMarker.type = visualization_msgs::Marker::MESH_RESOURCE;
        #MeshMarker.mesh_resource = "package://pr2_description/meshes/base_v0/base.dae";
    break


# Preprocess data

In [6]:
#Get mesh vertices and normals
mesh1 = meshstl.Mesh.from_file(('../'+ stl_file))
#mesh1 = meshstl.Mesh.from_file('../src/roboy_models/roboy_2_0_simplified/meshes/CAD/torso.stl')

global triangles 
triangles = scale * np.matrix(mesh1.points)


vertices = np.reshape(triangles,(len(triangles)*3,3)) 

#normals = np.zeros_like((len(triangles),3))
normalsNotNorm = mesh1.normals

normals = []
for normal in normalsNotNorm:
    normals.append(1/np.linalg.norm(normal,2)*normal)
    
normals = np.matrix(normals)
normals = scale * normals

#Initialize sensors in centers of triangle
sensors = (triangles[:,0:3]+triangles[:,3:6]+triangles[:,6:9])/3

print('%d triangles' %len(triangles))
#print(triangles)
print('')
print('%d vertices' %len(vertices))
#print(vertices)
print('')
print('%d normals normiert' %len(normals))
#print(normals)
print('')
print('%d sensors in centers of triangles' %len(sensors))
#print(sensors)
#print(vertices); print(normals); print(sensors)

20 triangles

60 vertices

20 normals normiert

20 sensors in centers of triangles


In [7]:
#Prepare vertices, normals and sensor locations for Transformation

verticesRot = np.insert(vertices, 3,values=1,axis=1)
#print(verticesRot)
#print('')

normalsRot = np.insert(normals, 3, values=0, axis=1)
#print(normalsRot)
#print('')

sensorsRot = np.insert(vertices, 3,values=1,axis=1)
#print(sensorsRot)
#print('')

In [8]:
#Test transform vertices
T = get_random_trafomatrix()
newvertices = np.dot(T,np.transpose(verticesRot)).T


#print(newvertices)

In [9]:
#Sensor visualization in RVIZ


for i,sensor in enumerate(sensors):

    sensor = np.squeeze(np.array(sensor))
    
    rate3 = rospy.Rate(10)
    while not rospy.is_shutdown():

        markers = Marker()
        markers.header.frame_id = 'world'
        markers.ns = 'ns'
        markers.type = Marker.SPHERE

        markers.pose.position.x = sensor[0];
        markers.pose.position.y = sensor[1];
        markers.pose.position.z = sensor[2];
        markers.pose.orientation.x = 0
        markers.pose.orientation.y = 0
        markers.pose.orientation.z = 0
        markers.pose.orientation.w = 0

        #print(marker.points)
        markers.color.r = 0.0
        markers.color.g = 0.0
        markers.color.b = 1.0
        markers.color.a = 1.0

        markers.scale.x = 0.01
        markers.scale.y = 0.01
        markers.scale.z = 0.01
        markers.lifetime = rospy.Duration(-1);
        markers.header.stamp = rospy.Time.now()
        markers.action = Marker.ADD
        markers.id = 100+i

        #mesh.mesh_resource = "package://common_utilities/stls/testcube_40mm.stl"
        #mesh.mesh_resource = "package://roboy_models/roboy_2_0_simplified/meshes/CAD/head.stl"

        publisher.publish(markers);
        rate3.sleep()
        break


# GA

In [10]:
from deap import algorithms, base, creator, tools

In [11]:
#sensors to dict
sensorList = []

for sensor in sensors:
    sensor = np.squeeze(np.array(sensor)).tolist()
    sensorList.append(sensor)

sensor_dict =  list(zip(range(len(sensors)), sensorList))

In [12]:
creator.create("FitnessMax", base.Fitness, weights=(1.0,1.0,1.0)) # 1 -> maximum probblem
creator.create("Individual", list, fitness=creator.FitnessMax)

In [13]:
toolbox = base.Toolbox()

In [14]:
# Find random sensors
def randomSensor(v):
    i = random.randint(0,len(v)-1)
    #print(i)
    rand = v[i]
    
    return rand[0]

In [15]:
r = np.squeeze(np.array(randomSensor(sensors)))

print(r)

[ 0.63146999  0.45878667  0.14907167]


In [16]:
toolbox = base.Toolbox()
# Attribute generator 
toolbox.register("attr_bool", randomSensor, sensor_dict)
# Structure initializers
toolbox.register("individual", tools.initRepeat, creator.Individual, 
    toolbox.attr_bool, 4)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

In [17]:
toolbox.attr_bool()

19

In [18]:
toolbox.individual()

[2, 6, 16, 19]

# Evaluation (Fitness) Function

In [19]:
from data.RayIntersectsTriangle import rayIntersectsTriangle,rayIntersectsTriangleVisualization

def FitnessFunction(sensors):

    br = tf.TransformBroadcaster()
    br.sendTransform((LH1[0][0], LH1[0][1], LH1[0][2]),
                     (quat1[0], quat1[1], quat1[2], quat1[3]),
                     rospy.Time.now(),
                     "lighthouse1",
                     "world")
    br.sendTransform((LH2[0][0], LH2[0][1], LH2[0][2]),
                     (quat2[0], quat2[1], quat2[2], quat2[3]),
                     rospy.Time.now(),
                     "lighthouse2",
                     "world")

    
    #1. COMPUTE VISIBLE SENSORS AT THE MOMENT
    LH1_array = np.asarray(LH1[0])
    LH2_array = np.asarray(LH2[0])
    #testTriangle = np.squeeze(np.asarray(triangles[0]))

    visibleLH1 = 0
    visibleLH2 = 0

    for i, nmbr_sensor in enumerate(sensors):
        sensor = sensor_dict[nmbr_sensor][1]

        #get distance of current sensor and check if intersection
        interDistLH1 = rayIntersectsTriangle(LH1_array, sensor, 
                                             np.squeeze(np.asarray(triangles[nmbr_sensor])), 'lighthouse1')
        interDistLH2 = rayIntersectsTriangle(LH2_array, sensor, 
                                             np.squeeze(np.asarray(triangles[nmbr_sensor])), 'lighthouse2')
        
        #print('interDist');print(interDistLH1);print(interDistLH2);print('endinterDist')

        for j in range(len(triangles)):
            if(nmbr_sensor != j):
                #print(j)
                newInterDistLH1 = rayIntersectsTriangle(LH1_array, sensor, 
                                                        np.squeeze(np.asarray(triangles[j])), 'lighthouse1')
                newInterDistLH2 = rayIntersectsTriangle(LH2_array, sensor, 
                                                        np.squeeze(np.asarray(triangles[j])), 'lighthouse2')
                if(newInterDistLH1 > interDistLH1 and newInterDistLH1 != False):
                    interDistLH1 = newInterDistLH1
                    visibleLH1 += 1

                if(newInterDistLH2 > interDistLH2 and newInterDistLH2 != False):
                    interDistLH2 = newInterDistLH2
                    visibleLH2 += 1
        #print(newInterDistLH1); print(newInterDistLH2)
        
    #2. COMPUTE EUCLIDEAN DISTANCE OF SENSORS
    individual = sensors
    dist = 0
    for i,ind in enumerate(individual):
        ind = np.asarray(sensor_dict[ind][1])
        for j in range(i,len(individual)):
            if(i != j):
                indivi = np.asarray(sensor_dict[individual[j]][1])
                dist += np.linalg.norm(ind-indivi)
    #print('visiblesensors')
    #print(visibleLH1);print(visibleLH2);print('')
    
    return dist, visibleLH1, visibleLH2
    #COST FUNCTION?
    #return dist, np.sqrt(visibleLH1**2 + visibleLH2**2)

In [20]:
toolbox.register("evaluate", FitnessFunction)

toolbox.register("mate", tools.cxTwoPoint)
# Independent probability  : for each attribute to be mutated.# low~up rondom int
toolbox.register("mutate", tools.mutUniformInt, low=0, up=len(sensorList)-1, indpb=0.2) 
toolbox.register("select", tools.selTournament, tournsize=3)

In [21]:
# Creating population

population = toolbox.population(n=30)

In [22]:
hof = tools.HallOfFame(1)

In [23]:
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("avg", np.mean)
stats.register("std", np.std)
stats.register("min", np.min)
stats.register("max", np.max)

In [24]:
population, log = algorithms.eaSimple(population, 
                                      toolbox, 
                                      cxpb=0.5, 
                                      mutpb=0.2, 
                                      ngen=10000, 
                                      stats=stats, 
                                      halloffame=hof, 
                                      verbose=True)

gen	nevals	avg    	std    	min	max    
0  	30    	2.89556	2.44648	0  	7.59286
1  	23    	3.31175	2.53696	0  	7.59288
2  	18    	3.33657	2.57582	0  	7.59286
3  	17    	3.42904	2.62509	0  	7.59286
4  	20    	3.50466	2.67314	0  	7.59288
5  	15    	3.50075	2.76444	1  	7.78593
6  	20    	3.57476	2.79066	1  	7.78593
7  	21    	3.65209	2.81377	1  	7.78593
8  	16    	3.80051	2.73761	1  	7.78593
9  	12    	3.90935	2.72644	1  	7.78593
10 	17    	3.86037	2.7213 	1  	7.78593
11 	20    	3.88948	2.70045	1  	7.78593
12 	13    	3.86575	2.74265	1  	7.78593
13 	14    	3.89915	2.71843	1  	7.78593
14 	23    	3.90182	2.70258	1  	7.78593
15 	14    	3.87238	2.72836	1  	7.78593
16 	17    	3.81249	2.64559	1  	7.78593
17 	13    	3.8972 	2.69389	1  	7.78593
18 	20    	3.8783 	2.6731 	1  	7.78593
19 	17    	3.91318	2.71181	1  	7.78593
20 	14    	3.8855 	2.711  	0  	7.78593
21 	18    	3.83678	2.70653	1  	7.78593
22 	15    	3.85069	2.68614	1  	7.78593
23 	21    	3.76975	2.6877 	0  	7.78593
24 	16    	3.91539	2.7344

210	17    	3.85485	2.69704	0  	7.78593
211	18    	3.88465	2.72419	1  	7.78593
212	19    	3.88771	2.72427	1  	7.78593
213	15    	3.90779	2.70936	1  	7.78593
214	22    	3.89408	2.73214	1  	7.78593
215	18    	3.86518	2.70159	1  	7.78593
216	18    	3.91493	2.69475	1  	7.78593
217	17    	3.85284	2.69328	1  	7.78593
218	19    	3.90097	2.71733	1  	7.78593
219	24    	3.8865 	2.68106	1  	7.78593
220	17    	3.90871	2.69138	1  	7.78593
221	16    	3.88718	2.68301	1  	7.78593
222	23    	3.89909	2.71763	1  	7.78593
223	15    	3.85789	2.71786	1  	7.78593
224	20    	3.90331	2.71844	1  	7.78593
225	18    	3.85847	2.73343	1  	7.78593
226	20    	3.92222	2.6798 	2  	7.78593
227	23    	3.88491	2.72035	1  	7.78593
228	16    	3.90364	2.70471	1  	7.78593
229	23    	3.80878	2.66583	0  	7.78593
230	20    	3.89655	2.69268	1  	7.78593
231	11    	3.91233	2.73044	1  	7.78593
232	16    	3.8464 	2.68525	1  	7.78593
233	16    	3.88467	2.7005 	1  	7.78593
234	15    	3.89408	2.73214	1  	7.78593
235	15    	3.84282	2.6953

422	16    	3.89915	2.71843	1  	7.78593
423	21    	3.90954	2.69243	1  	7.78593
424	13    	3.92534	2.70464	2  	7.78593
425	18    	3.8618 	2.68013	1  	7.78593
426	20    	3.9174 	2.71669	1  	7.78593
427	15    	3.91018	2.72747	1  	7.78593
428	20    	3.91862	2.67803	1  	7.78593
429	14    	3.88798	2.72332	1  	7.78593
430	13    	3.8603 	2.71694	1  	7.78593
431	21    	3.91111	2.69393	1  	7.78593
432	12    	3.90488	2.68345	1  	7.78593
433	15    	3.91532	2.73432	1  	7.78593
434	20    	3.91144	2.67416	2  	7.78593
435	21    	3.89303	2.72926	1  	7.78593
436	13    	3.91234	2.71077	1  	7.78593
437	17    	3.91416	2.71362	1  	7.78593
438	15    	3.86849	2.73279	0  	7.78593
439	13    	3.87231	2.71306	1  	7.78593
440	14    	3.81575	2.64362	1  	7.78593
441	15    	3.90629	2.72246	1  	7.78593
442	16    	3.90097	2.71733	1  	7.78593
443	13    	3.89668	2.71549	1  	7.78593
444	12    	3.89362	2.71245	1  	7.78593
445	26    	3.89344	2.6919 	1  	7.78593
446	19    	3.90084	2.70076	1  	7.78593
447	15    	3.90239	2.7196

633	18    	3.93372	2.71372	2  	7.78593
634	12    	3.89485	2.73184	1  	7.78593
635	17    	3.91149	2.72942	1  	7.78593
636	26    	3.92404	2.69212	1  	7.78593
637	15    	3.902  	2.72057	1  	7.78593
638	14    	3.90272	2.72313	1  	7.78593
639	17    	3.86009	2.69954	1  	7.78593
640	11    	3.92138	2.70114	2  	7.78593
641	14    	3.9233 	2.70083	2  	7.78593
642	18    	3.87316	2.70883	1  	7.78593
643	24    	3.90484	2.70098	1  	7.78593
644	21    	3.9265 	2.72455	2  	7.78593
645	16    	3.85011	2.71105	0  	7.78593
646	19    	3.77162	2.73328	0  	7.78593
647	22    	3.87013	2.66334	1  	7.78593
648	22    	3.82585	2.73752	1  	7.78593
649	13    	3.86959	2.6866 	1  	7.78593
650	18    	3.90423	2.70111	1  	7.78593
651	15    	3.89279	2.71052	1  	7.78593
652	21    	3.8143 	2.70693	1  	7.78593
653	16    	3.92604	2.68475	2  	7.78593
654	23    	3.87986	2.72166	1  	7.78593
655	19    	3.90727	2.72455	1  	7.78593
656	16    	3.89545	2.69317	1  	7.78593
657	10    	3.92514	2.68689	1  	7.78593
658	19    	3.86498	2.6817

844	18    	3.90298	2.71858	1  	7.78593
845	17    	3.87128	2.69373	1  	7.78593
846	15    	3.82476	2.71539	1  	7.78593
847	24    	3.88337	2.68929	1  	7.78593
848	16    	3.84704	2.72076	1  	7.78593
849	23    	3.84614	2.69925	1  	7.78593
850	18    	3.90448	2.69566	1  	7.78593
851	13    	3.87356	2.71408	1  	7.78593
852	24    	3.85816	2.71363	1  	7.78593
853	18    	3.785  	2.68995	0  	7.78593
854	18    	3.83387	2.68636	0  	7.78593
855	21    	3.859  	2.70079	1  	7.78593
856	18    	3.85841	2.69906	1  	7.78593
857	20    	3.89915	2.71843	1  	7.78593
858	17    	3.87841	2.69697	1  	7.78593
859	9     	3.88039	2.66183	1  	7.78593
860	18    	3.8486 	2.70996	0  	7.78593
861	16    	3.8343 	2.729  	1  	7.78593
862	19    	3.90213	2.74122	1  	7.78593
863	16    	3.88195	2.65588	1  	7.78593
864	19    	3.8332 	2.72964	1  	7.78593
865	15    	3.88628	2.70117	1  	7.78593
866	25    	3.90599	2.65906	1  	7.78593
867	19    	3.91507	2.69567	1  	7.78593
868	15    	3.88747	2.68618	1  	7.78593
869	21    	3.89576	2.7144

...

9076	22    	3.88385	2.71833	1  	7.78593
9077	27    	3.89578	2.69478	1  	7.78593
9078	21    	3.91838	2.71469	2  	7.78593
9079	14    	3.86303	2.70152	1  	7.78593
9080	21    	3.72848	2.71292	0  	7.78593
9081	20    	3.83937	2.70965	0  	7.78593
9082	25    	3.88655	2.70418	1  	7.78593
9083	23    	3.83989	2.71594	1  	7.78593
9084	18    	3.83789	2.66877	1  	7.78593
9085	19    	3.84282	2.70745	0  	7.78593
9086	18    	3.89337	2.69635	1  	7.78593
9087	18    	3.76514	2.70052	0  	7.78593
9088	16    	3.84599	2.68671	1  	7.78593
9089	16    	3.85035	2.72993	0  	7.78593
9090	19    	3.86295	2.72537	0  	7.78593
9091	18    	3.87206	2.65209	1  	7.78593
9092	17    	3.87485	2.73176	1  	7.78593
9093	17    	3.87759	2.71286	1  	7.78593
9094	18    	3.81878	2.72584	0  	7.78593
9095	14    	3.88899	2.72814	1  	7.78593
9096	22    	3.856  	2.71992	0  	7.78593
9097	20    	3.88403	2.66085	1  	7.78593
9098	22    	3.90396	2.70162	1  	7.78593
9099	23    	3.90298	2.69951	1  	7.78593
9100	18    	3.87601	2.69621	1  	7.78593


9281	22    	3.84061	2.68549	1  	7.78593
9282	9     	3.9265 	2.72455	2  	7.78593
9283	6     	3.84328	2.72002	1  	7.78593
9284	21    	3.91409	2.69476	1  	7.78593
9285	16    	3.92864	2.72751	2  	7.78593
9286	23    	3.88967	2.66076	1  	7.78593
9287	17    	3.90954	2.69243	1  	7.78593
9288	17    	3.80438	2.69978	0  	7.78593
9289	21    	3.79148	2.68814	1  	7.78593
9290	19    	3.8354 	2.69329	1  	7.78593
9291	20    	3.80617	2.73898	1  	7.78593
9292	22    	3.84523	2.72037	1  	7.78593
9293	14    	3.9063 	2.70378	1  	7.78593
9294	22    	3.88905	2.66197	1  	7.78593
9295	21    	3.90895	2.70706	1  	7.78593
9296	20    	3.84027	2.73324	1  	7.78593
9297	24    	3.86653	2.71994	1  	7.78593
9298	23    	3.86555	2.70038	1  	7.78593
9299	18    	3.90922	2.7064 	1  	7.78593
9300	16    	3.90629	2.72246	1  	7.78593
9301	14    	3.8768 	2.71438	1  	7.78593
9302	25    	3.85329	2.65668	0  	7.78593
9303	20    	3.87622	2.69392	1  	7.78593
9304	14    	3.89357	2.69166	1  	7.78593
9305	16    	3.92818	2.70679	2  	7.78593


9488	16    	3.913  	2.69361	1  	7.78593
9489	19    	3.87653	2.6751 	1  	7.78593
9490	19    	3.86992	2.70004	0  	7.78593
9491	15    	3.92949	2.70886	2  	7.78593
9492	20    	3.85011	2.70472	1  	7.78593
9493	16    	3.8855 	2.72446	1  	7.78593
9494	15    	3.90311	2.72032	1  	7.78593
9495	16    	3.83479	2.68455	0  	7.78593
9496	22    	3.89252	2.71208	1  	7.78593
9497	16    	3.88069	2.71836	1  	7.78593
9498	19    	3.89909	2.71763	1  	7.78593
9499	20    	3.85497	2.64973	1  	7.78593
9500	24    	3.79768	2.67105	0  	7.78593
9501	14    	3.92344	2.72059	2  	7.78593
9502	17    	3.88161	2.70147	1  	7.78593
9503	15    	3.9135 	2.71388	1  	7.78593
9504	13    	3.87218	2.70943	1  	7.78593
9505	14    	3.90928	2.7079 	1  	7.78593
9506	18    	3.8805 	2.73852	1  	7.78593
9507	15    	3.8811 	2.68174	1  	7.78593
9508	19    	3.80222	2.68547	0  	7.78593
9509	18    	3.80736	2.68524	1  	7.78593
9510	18    	3.90513	2.72157	1  	7.78593
9511	19    	3.85945	2.73553	1  	7.78593
9512	17    	3.86886	2.73079	0  	7.78593


9693	25    	3.89279	2.71052	1  	7.78593
9694	15    	3.89486	2.69736	1  	7.78593
9695	25    	3.89336	2.73002	1  	7.78593
9696	19    	3.88233	2.70174	1  	7.78593
9697	15    	3.88427	2.68532	1  	7.78593
9698	19    	3.83932	2.69662	1  	7.78593
9699	13    	3.85614	2.71746	0  	7.78593
9700	22    	3.90715	2.66609	1  	7.78593
9701	19    	3.86569	2.72094	1  	7.78593
9702	21    	3.84777	2.70283	1  	7.78593
9703	14    	3.9226 	2.72364	1  	7.78593
9704	18    	3.81124	2.72755	0  	7.78593
9705	16    	3.89611	2.69036	1  	7.78593
9706	19    	3.87368	2.65175	1  	7.78593
9707	13    	3.93645	2.69875	2  	7.78593
9708	18    	3.85367	2.68839	1  	7.78593
9709	24    	3.90052	2.68847	1  	7.78593
9710	22    	3.80597	2.69621	1  	7.78593
9711	20    	3.86297	2.70935	0  	7.78593
9712	22    	3.86245	2.71968	1  	7.78593
9713	21    	3.81658	2.66157	1  	7.78593
9714	21    	3.87537	2.71267	1  	7.78593
9715	13    	3.89824	2.73622	1  	7.78593
9716	17    	3.92779	2.6906 	1  	7.78593
9717	22    	3.82152	2.70559	0  	7.78593


9898	18    	3.89824	2.73622	1  	7.78593
9899	20    	3.89187	2.72852	1  	7.78593
9900	24    	3.8976 	2.69619	1  	7.78593
9901	15    	3.88367	2.72223	1  	7.78593
9902	20    	3.90136	2.67758	1  	7.78593
9903	22    	3.90792	2.68503	1  	7.78593
9904	19    	3.91941	2.69584	2  	7.78593
9905	21    	3.9189 	2.68071	1  	7.78593
9906	26    	3.78638	2.69454	1  	7.78593
9907	16    	3.87673	2.70899	1  	7.78593
9908	18    	3.86146	2.73666	1  	7.78593
9909	15    	3.88804	2.72819	1  	7.78593
9910	16    	3.87835	2.73554	1  	7.78593
9911	21    	3.79117	2.66464	1  	7.78593
9912	17    	3.85192	2.71909	0  	7.78593
9913	21    	3.90519	2.72236	1  	7.78593
9914	17    	3.88467	2.70476	1  	7.78593
9915	15    	3.87823	2.67319	1  	7.78593
9916	19    	3.88635	2.68229	1  	7.78593
9917	22    	3.92864	2.72751	2  	7.78593
9918	16    	3.89735	2.67205	2  	7.78593
9919	16    	3.92864	2.72751	2  	7.78593
9920	17    	3.89311	2.71505	1  	7.78593
9921	22    	3.854  	2.71451	1  	7.78593
9922	19    	3.87841	2.69947	1  	7.78593


In [30]:
best_sensors = tools.selBest(population, k=5)
print(best_sensors)

[[14, 0, 16, 8], [14, 0, 16, 8], [14, 0, 16, 8], [14, 0, 16, 8], [14, 0, 16, 8]]


In [31]:
#Sensor visualization in RVIZ

for i, best in enumerate(best_sensors[0]):

    show_sensor = sensor_dict[best][1]
    
    rate3 = rospy.Rate(10)
    while not rospy.is_shutdown():

        markers = Marker()
        markers.header.frame_id = 'world'
        markers.ns = 'ns'
        markers.type = Marker.SPHERE

        markers.pose.position.x = show_sensor[0];
        markers.pose.position.y = show_sensor[1];
        markers.pose.position.z = show_sensor[2];
        markers.pose.orientation.x = 0
        markers.pose.orientation.y = 0
        markers.pose.orientation.z = 0
        markers.pose.orientation.w = 0

        #print(marker.points)
        markers.color.r = 1.0
        markers.color.g = 0.0
        markers.color.b = 0.0
        markers.color.a = 1.0

        markers.scale.x = 0.015
        markers.scale.y = 0.015
        markers.scale.z = 0.015
        markers.lifetime = rospy.Duration(-1);
        markers.header.stamp = rospy.Time.now()
        markers.action = Marker.ADD
        markers.id = 100+i


        publisher.publish(markers);
        rate3.sleep()
        break
